In [1]:
import os
import numpy as np
import scipy.stats
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("/SNS/users/m2d/MR/notebooks")
from plot_utils import plot1d, plot_heatmap

In [3]:
from scipy import ndimage
import scipy.signal

In [14]:
def plot_slice(file_list, index=75, q_axis=True, width=0):
    _plot_list = []
    _plot_names = []
    
    for i, f in enumerate(file_list):
        _data, _, _, _ = load_data(f, index=index, q_axis=q_axis, width=width)
        
        if q_axis and os.path.isfile(f.replace("OffSpecSmooth", "Specular")):
            spec_data = np.loadtxt(f.replace("OffSpecSmooth", "Specular"))
            
            _plot_list.append([spec_data.T[0], spec_data.T[1], spec_data.T[2]])
            _plot_names.append("Spec v%s" % str(i+1))
        
        _plot_list.append(_data)
        _plot_names.append("v%s" % str(i+1))
    
    plot1d(_plot_list, _plot_names,
           x_title='', y_title='',
           x_log=False, y_log=True, show_dx=True)

    
def load_data(file_path, index=75, q_axis=True, width=0):
    offspec_data = np.loadtxt(file_path)

    unique_x = np.unique(offspec_data.T[0])
    xbins = len(unique_x)
    unique_x = np.unique(offspec_data.T[1])
    zbins = len(unique_x)

    print("\n\nFile: %s" % file_path)
    print("Binning: %s %s [%s]" %(xbins, zbins, xbins*zbins))

    off = np.reshape(offspec_data, (zbins, xbins, 3))

    dk = (off.T[0]).T[0]
    qz = (off.T[1])[0]

    quick_intensity = off.T[2].T
    
    if q_axis:
        if width>0:
            refl = np.sum(quick_intensity.T[index-width:index+width+1], axis=0)
            refl /= 10.0
        else:
            refl = quick_intensity.T[index]
        _index_value = dk[index]
    else:
        refl = quick_intensity[index]
        _index_value = qz[index]

    scale = np.max(quick_intensity)
    #scale = np.max(refl)
    #scale = 1
    quick_intensity /= scale
    quick_intensity[quick_intensity<1.0e-9] = 1.0e-9

    print("Scale = %s" % scale)
    print("Const value = %s" % _index_value)
    print("Width = %s" % width)
    return [qz, refl/scale], dk, qz, quick_intensity


In [15]:
# Center 2 are 41 and 42

plot_slice(['/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat',
            '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat'],
          index=42, q_axis=True, width=5)




File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat
Binning: 84 282 [23688]
Scale = 3.602153
Const value = 0.0001807229
Width = 5


File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat
Binning: 84 282 [23688]
Scale = 3.05251
Const value = 0.000180723
Width = 5


# Old QuickNXS v1 version

In [16]:
_, dk, qz, off_spec = load_data('/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat')

plot_heatmap(dk, qz, np.log(off_spec), x_title='', y_title='', surface=False,
                 x_log=False, y_log=False)



File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat
Binning: 84 282 [23688]
Scale = 3.602153
Const value = 0.01210843
Width = 0


# QuickNXS v2 output

In [17]:
_, dk, qz, off_spec = load_data('/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat')

plot_heatmap(dk, qz, np.log(off_spec), x_title='', y_title='', surface=False,
                 x_log=False, y_log=False)



File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off.dat
Binning: 84 282 [23688]
Scale = 3.05251
Const value = 0.0121084
Width = 0
